In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_hero
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
WITH asm_kg_tv AS (
  SELECT DISTINCT
    st.business_unit,
    a.store_id,
    a.product_id
  FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_scd a
  LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store st USING (store_id)
  WHERE
    (
      (st.business_unit = '2000' AND a.assortment IN ('VMP_KGMB', 'VMP_KGMN', 'WMP_THOIVU')) OR
      (st.business_unit = '1500' AND (a.assortment LIKE 'KG\\_%' OR a.assortment LIKE 'VM\\_KG%')) OR
      a.assortment LIKE 'KG_%'
    )
    AND DATE(start_date) <= DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
    AND DATE(end_date) >= DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
    AND a.status = 'ACTIVE'
)
SELECT DISTINCT
  a2.mien,
  a2.business_unit,
  a2.city,
  a.store_id,
  a2.store_name,
  a.product_id,
  a1.product_name,
  a1.base_uom,
  a1.manufacturer_id,
  a1.manufacturer_name,
  a1.mch2_id,
  a1.mch3_id,
  a1.mch4_id,
  a1.mch5_id,
  a.assortment,
  CASE
    WHEN a.assortment LIKE '%HERO1' THEN 'HERO1'
    WHEN a.assortment LIKE '%HERO2' THEN 'HERO2'
    WHEN a.assortment LIKE '%HERO' THEN 'HERO'
  END AS assortment_class
FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_scd a
LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product a1 USING (product_id)
LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store a2 USING (store_id)
LEFT JOIN asm_kg_tv a3 USING (store_id, product_id)
WHERE
  a.end_date >= DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
  AND a.status = 'ACTIVE'
  AND a3.product_id IS NULL
  AND ( a.assortment LIKE '%HERO%' OR a.assortment LIKE '%HERO')
""")